In [78]:
# importing necessary libraries
import pandas as pd
import datetime as dt
import numpy as np
 
# importing the data
df = pd.read_csv("orders.csv", decimal= '.')
df.head()

,CustomerID,Customer Status,Date Order was placed,Delivery Date,Order ID,Product ID,Quantity Ordered,Total Retail Price for This Order,Cost Price Per Unit
0,579,Silver,01-Jan-17,07-Jan-17,123002578,220101400106,2,92.6,20.70
1,7574,SILVER,01-Jan-17,05-Jan-17,123004074,210201000009,1,21.7,9.95
2,28861,Gold,01-Jan-17,04-Jan-17,123000871,230100500068,1,1.7,0.80
3,43796,Gold,01-Jan-17,06-Jan-17,123002851,220100100633,1,47.9,24.05
4,54673,Gold,01-Jan-17,04-Jan-17,123003607,220200200043,1,36.9,18.30


In [80]:
df['Date Order was placed'] = pd.to_datetime(df['Date Order was placed'])

df_recency = df.groupby(by='CustomerID', 
                        as_index=False)['Date Order was placed'].max()
df_recency.columns = ['CustomerID', 'LastPurchaseDate']
recent_date = df_recency['LastPurchaseDate'].max()
df_recency['Recency'] = df_recency['LastPurchaseDate'].apply(lambda x: (recent_date - x).days)
df_recency.head()

,CustomerID,LastPurchaseDate,Recency
0,1,2021-10-08,84
1,3,2021-04-11,264
2,4,2018-08-19,1230
3,5,2021-05-02,243
4,6,2021-06-22,192


In [82]:
frequency_df = df.drop_duplicates().groupby(
    by=['CustomerID'], as_index=False)['Date Order was placed'].count()
frequency_df.columns = ['CustomerID', 'Frequency']
frequency_df.head()

,CustomerID,Frequency
0,1,7
1,3,3
2,4,2
3,5,5
4,6,3


In [84]:
df['Total'] = df['Cost Price Per Unit']*df['Quantity Ordered']
monetary_df = df.groupby(by='CustomerID', as_index=False)['Total'].sum()
monetary_df.columns = ['CustomerID', 'Monetary']
monetary_df.head()

,CustomerID,Monetary
0,1,305.5
1,3,172.2
2,4,179.0
3,5,272.8
4,6,168.2


In [85]:
rf_df = df_recency.merge(frequency_df, on='CustomerID')
rfm_df = rf_df.merge(monetary_df, on='CustomerID').drop(
    columns='LastPurchaseDate')
rfm_df.head()

,CustomerID,Recency,Frequency,Monetary
0,1,84,7,305.5
1,3,264,3,172.2
2,4,1230,2,179.0
3,5,243,5,272.8
4,6,192,3,168.2


In [86]:
# Define scoring criteria for each RFM value
recency_scores = [5, 4, 3, 2, 1]  # Higher score for lower recency (more recent)
frequency_scores = [1, 2, 3, 4, 5]  # Higher score for higher frequency
monetary_scores = [1, 2, 3, 4, 5]  # Higher score for higher monetary value

# Calculate RFM scores
rfm_df['RecencyScore'] = pd.cut(rfm_df['Recency'], bins=5, labels=recency_scores)
rfm_df['FrequencyScore'] = pd.cut(rfm_df['Frequency'], bins=5, labels=frequency_scores)
rfm_df['MonetaryScore'] = pd.cut(rfm_df['Monetary'], bins=5, labels=monetary_scores)

In [87]:
# Convert RFM scores to numeric type
rfm_df['RecencyScore'] = rfm_df['RecencyScore'].astype(int)
rfm_df['FrequencyScore'] = rfm_df['FrequencyScore'].astype(int)
rfm_df['MonetaryScore'] = rfm_df['MonetaryScore'].astype(int)

In [88]:
# Calculate RFM score by combining the individual scores
rfm_df['RFM_Score'] = rfm_df['RecencyScore'] + rfm_df['FrequencyScore'] + rfm_df['MonetaryScore']

# Create RFM segments based on the RFM score
segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']
rfm_df['Value Segment'] = pd.qcut(rfm_df['RFM_Score'], q=3, labels=segment_labels)

In [89]:
print(rfm_df.head())

   CustomerID  Recency  Frequency  Monetary  RecencyScore  FrequencyScore  \
0           1       84          7     305.5             5               2   
1           3      264          3     172.2             5               1   
2           4     1230          2     179.0             2               1   
3           5      243          5     272.8             5               1   
4           6      192          3     168.2             5               1   

   MonetaryScore  RFM_Score Value Segment  
0              1          8    High-Value  
1              1          7     Mid-Value  
2              1          4     Low-Value  
3              1          7     Mid-Value  
4              1          7     Mid-Value  


In [90]:
# Create a new column for RFM Customer Segments
rfm_df['RFM Customer Segments'] = ''

# Assign RFM segments based on the RFM score
rfm_df.loc[rfm_df['RFM_Score'] >= 9, 'RFM Customer Segments'] = 'Champions'
rfm_df.loc[(rfm_df['RFM_Score'] >= 6) & (rfm_df['RFM_Score'] < 9), 'RFM Customer Segments'] = 'Potential Loyalists'
rfm_df.loc[(rfm_df['RFM_Score'] >= 5) & (rfm_df['RFM_Score'] < 6), 'RFM Customer Segments'] = 'At Risk Customers'
rfm_df.loc[(rfm_df['RFM_Score'] >= 4) & (rfm_df['RFM_Score'] < 5), 'RFM Customer Segments'] = "Can't Lose"
rfm_df.loc[(rfm_df['RFM_Score'] >= 3) & (rfm_df['RFM_Score'] < 4), 'RFM Customer Segments'] = "Lost"

# Print the updated data with RFM segments
print(rfm_df[['CustomerID', 'RFM Customer Segments']])


       CustomerID RFM Customer Segments
0               1   Potential Loyalists
1               3   Potential Loyalists
2               4            Can't Lose
3               5   Potential Loyalists
4               6   Potential Loyalists
...           ...                   ...
56022       94246   Potential Loyalists
56023       94247     At Risk Customers
56024       94250     At Risk Customers
56025       94252   Potential Loyalists
56026       94253            Can't Lose

[56027 rows x 2 columns]


In [91]:
# Verwijder niet-numerieke karakters (behalve de decimaalteken) uit de 'Monetary' kolom
rfm_df['Monetary'] = rfm_df['Monetary'].replace('[^\d.]', '', regex=True).astype(float)
# Converteer de 'Monetary' kolom naar float
rfm_df['Monetary'] = rfm_df['Monetary'].astype(float)

In [68]:
df = pd.DataFrame(rfm_df)

In [69]:
df.to_csv('rfm_df.csv', index=False, sep = ",", decimal= ",")